In [1]:
# check https://github.com/Tomiinek/Multilingual_Text_to_Speech/blob/master/synthesize.py
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import json

SetLogLevel(0)

# Read audiofile


In [2]:
model_vosk = Model("model")

In [3]:
%%time

wf = wave.open("untitled.wav", "rb")
if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
    print ("Audio file must be WAV format mono PCM.")
    exit (1)

rec = KaldiRecognizer(model_vosk, wf.getframerate())

results = []

while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        #print(rec.Result())
        jj = json.loads(rec.Result())["text"]
        print(jj)
        results.append( jj )
        pass
    else:
        #print(rec.PartialResult())
        pass

#print(rec.FinalResult())
j = json.loads( rec.FinalResult() )
print(j['text'])
for r in results:
    print(r)

tiens et à opposer la figure de mme de maintenon à celle de ninon de lenclos d'un côté la femme de lettres galantes et libertines
et de l autre la femme de pouvoir secrète et des votes
le mythe du grand siècle dont nous sommes les héritiers articule ainsi une image de l'écriture féminine
tiens et à opposer la figure de mme de maintenon à celle de ninon de lenclos d'un côté la femme de lettres galantes et libertines
et de l autre la femme de pouvoir secrète et des votes
CPU times: user 3.39 s, sys: 14.2 ms, total: 3.41 s
Wall time: 3.4 s


# load GPT-2 model 

* load gpt model from our checkpoint
* load tokenizer from our checkpoint
* put it on the gpu
* set it to eval mode

In [8]:
# https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh?usp=sharing#scrollTo=hQ1oK0kXaV5p
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

# Load pretrained model and tokenizer
model_test = GPT2LMHeadModel.from_pretrained("results-finetune/checkpoint-2500/")
tokenizer = GPT2TokenizerFast.from_pretrained("antoiloui/belgpt2", model_max_length=768, pad_token='<|pad|>')

#device = torch.device("cuda")

#model_test.cuda()

# Generate a sample of text
model_test.eval()

model_test.resize_token_embeddings(len(tokenizer))


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Embedding(50259, 768)

# make prompt from vosk results

In [9]:
from collections import deque
print(results)

["tiens et à opposer la figure de mme de maintenon à celle de ninon de lenclos d'un côté la femme de lettres galantes et libertines", 'et de l autre la femme de pouvoir secrète et des votes']


In [19]:



aa = deque(maxlen=10)
for_wave_rnn = []

for q in results:
    prompt = "{}A: {}\n B: ".format(tokenizer.bos_token, q)
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    output = model_test.generate(
                inputs,
                do_sample=True,   
                top_k=50, 
                max_length=len(inputs) + 100,
                top_p=0.55, 
                num_return_sequences=2,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.65
    )

    # Decode it
    decoded_output = []
    for sample in output:
        decoded_output.append(tokenizer.decode(sample, skip_special_tokens=True))
    print(decoded_output)
   
    for o in decoded_output:
        b = o.split( 'B:' )[1]
        #print(b)
        for_wave_rnn.append(b + "|00-de|fr")
        
    ll = sorted(for_wave_rnn, key=len)
    aa.appendleft(ll[-1])
    print(ll[-1])



["A: tiens et à opposer la figure de mme de maintenon à celle de ninon de lenclos d'un côté la femme de lettres galantes et libertines\n B:  (À B, avec douceur) C’est vrai que notre vie a basculé, comme ça, dans l’intérieur!", "A: tiens et à opposer la figure de mme de maintenon à celle de ninon de lenclos d'un côté la femme de lettres galantes et libertines\n B:  (À tout le PEUPLE réuni.) Eh bien, ma chère A, je te demande pardon, mais je ne veux pas te punir. (Pause)"]
  (À tout le PEUPLE réuni.) Eh bien, ma chère A, je te demande pardon, mais je ne veux pas te punir. (Pause)|00-de|fr
['A: et de l autre la femme de pouvoir secrète et des votes\n B:  Oui. Mais le mariage ne devait pas avoir d’éclair.', 'A: et de l autre la femme de pouvoir secrète et des votes\n B:  (Enlève la main couvrant son vide) C’est vrai que tu as beaucoup de respect pour ton mari et pour ton fils. Tu es très polie!']
  (Enlève la main couvrant son vide) C’est vrai que tu as beaucoup de respect pour ton mari et

In [20]:
print(aa)

deque(['  (Enlève la main couvrant son vide) C’est vrai que tu as beaucoup de respect pour ton mari et pour ton fils. Tu es très polie!|00-de|fr', '  (À tout le PEUPLE réuni.) Eh bien, ma chère A, je te demande pardon, mais je ne veux pas te punir. (Pause)|00-de|fr'], maxlen=10)


# load wavernn and taco model

In [6]:
os.chdir(os.path.expanduser("~"))
    
tacotron_dir = "Multilingual_Text_to_Speech"
wavernn_dir = "WaveRNN"

os.chdir(os.path.join(os.path.expanduser("~"), "checkpoints"))

tacotron_chpt = "generated_switching.pyt"
wavernn_chpt = "wavernn_weight.pyt"

os.chdir(os.path.expanduser("~"))

os.chdir(os.path.join(os.path.expanduser("~"), tacotron_dir))
if "utils" in sys.modules: del sys.modules["utils"]

from synthesize import synthesize
from params.params import Params as hp
from utils import build_model, audio

model_taco = build_model(os.path.join(os.path.expanduser("~"), "checkpoints", tacotron_chpt))
model_taco.eval()
model_taco.cuda()

os.chdir(os.path.join(os.path.expanduser("~"), wavernn_dir))
if "utils" in sys.modules: del sys.modules["utils"]

from models.fatchord_version import WaveRNN
from utils import hparams as hp
from gen_wavernn import generate
import torch

hp.configure('hparams.py')
# model_wave = WaveRNN(rnn_dims=hp.voc_rnn_dims, fc_dims=hp.voc_fc_dims, bits=hp.bits, pad=hp.voc_pad, upsample_factors=hp.voc_upsample_factors, 
#                feat_dims=hp.num_mels, compute_dims=hp.voc_compute_dims, res_out_dims=hp.voc_res_out_dims, res_blocks=hp.voc_res_blocks, 
#                hop_length=hp.hop_length, sample_rate=hp.sample_rate, mode=hp.voc_mode).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
#model_wave.load(os.path.join(os.path.expanduser("~"), "checkpoints", wavernn_chpt))




/home/datascience/WaveRNN/utils/display.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/anaconda/envs/py37_pytorch/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/envs/py37_pytorch/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 612, in start
    se

# do the heavy work and generate sound

In [26]:
%%time
import IPython
from params.params import Params as hp
#spectrograms = [synthesize(model_taco, "|" + i) for i in for_wave_rnn]
#waveforms = [generate(model_wave, s, hp.voc_gen_batched, hp.voc_target, hp.voc_overlap) for s in spectrograms]
#waveforms = [audio.inverse_spectrogram(s, not hp.predict_linear) for s in spectrograms]

a = synthesize(model_taco, "|" + ll[-1])
b = audio.inverse_spectrogram(a, not hp.predict_linear)

print(ll[-1])
IPython.display.display(IPython.display.Audio(data=b, rate=hp.sample_rate))

#for idx, w in enumerate(waveforms):
#  print(for_wave_rnn[idx])
#  IPython.display.display(IPython.display.Audio(data=w, rate=hp.sample_rate))

  (À tout le PEUPLE réuni.) Eh bien, je crois que nous avons tout à gagner en faisant se rencontrer deux ou trois de ces personnes qui sont en train de s’écrouler dans une maison différente…|00-de|fr


CPU times: user 8.12 s, sys: 156 ms, total: 8.28 s
Wall time: 5.97 s


In [8]:
import librosa

In [30]:
print(hp.sample_rate)

22050
